In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import recall_score, roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score, roc_auc_score, make_scorer

from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier

from tqdm import tqdm
from functools import reduce
from copy import deepcopy
import gc
from collections import Counter

%matplotlib inline

import warnings
warnings.filterwarnings('ignore') 

%cd ../

/home/chervovn04/Programming/hackathons/2022/digital_breakout_885303


In [2]:
def delete_double_spaces(nm):
    new_nm = ''
    for char in nm:
        if char != ' ' or (len(new_nm) and new_nm[-1] != ' '):
            new_nm += char
    return new_nm.strip()


def label_process(nm):
    nm = nm.lower()
    new_nm = ''
    balance = 0
    for char in nm:
        if char in '({[':
            balance += 1
        elif char in ')}]':
            balance = max(0, balance - 1)
        elif balance == 0:
            new_nm += char
    nm = new_nm
    nm = delete_double_spaces(nm)
    return nm

In [3]:
df = pd.read_csv("data/train.csv")
test_data = pd.read_csv("data/test.csv")
test_ids = deepcopy(test_data.id)

In [4]:
df['processed_name'] = df.name_mfi.apply(label_process)
test_data['processed_name'] = test_data.name_mfi.apply(label_process)

In [5]:
df['item_hash'] = df['name_mfi'].apply(hash) ^ df.weight_mfi.apply(lambda x: hash(str(x))) ^ df.weight.apply(lambda x: hash(str(x))) ^ \
    df.mailtype.apply(lambda x: hash(str(x))) ^ df.transport_pay.apply(lambda x: hash(str(x)))
test_data['item_hash'] = test_data['item_hash'] = test_data['name_mfi'].apply(hash) ^ test_data.weight_mfi.apply(lambda x: hash(str(x))) ^ test_data.weight.apply(lambda x: hash(str(x))) ^ \
    test_data.mailtype.apply(lambda x: hash(str(x))) ^ test_data.transport_pay.apply(lambda x: hash(str(x)))

df['oper_item_hash'] = df.index_oper.apply(lambda x: hash(str(x))) ^ df.item_hash
test_data['oper_item_hash'] = test_data.index_oper.apply(lambda x: hash(str(x))) ^ test_data.item_hash

In [6]:
hash_counter = Counter(pd.concat([df.item_hash, test_data.item_hash], ignore_index=True))
oper_item_hash_counter = Counter(pd.concat([df.oper_item_hash, test_data.oper_item_hash], ignore_index=True))
name_counter = Counter(pd.concat([df.processed_name, test_data.processed_name], ignore_index=True))

In [7]:
id_hash = pd.concat([df[['id', 'item_hash']], test_data[['id', 'item_hash']]], ignore_index=True)
id_hash = id_hash.groupby('item_hash').agg(max).rename(columns={'id': 'last_id'})

In [8]:
df = df.join(id_hash, on='item_hash')
test_data = test_data.join(id_hash, on='item_hash')

In [9]:
X = df.drop(columns=["label"])
y = df.label
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)

In [10]:
top_operators = [obj for obj, count in Counter(test_data['index_oper'].replace(' ', '0').astype(float).astype(int)).most_common()][:10]

In [11]:
useless = []
cat_features = ["type", "is_privatecategory", "class", "is_in_yandex", "mailctg", "directctg", "mailtype"]
cat_features = list(set(cat_features) - set(useless))

def prepare_data(df):
    oper_type, oper_attr = zip(*df['oper_type + oper_attr'].map(lambda x: x.split('_')))
    
    # df['is_negative_oper_attr'] = pd.Series(oper_attr) == '-1'
    df['specialization_ratio'] = df.total_qty_over_index_and_type / df.total_qty_over_index.clip(lower=1)
    df['total_over_person'] = df.total_qty_over_index / df.dist_qty_oper_login_1.clip(lower=1)
    df['unknown_oper_ratio'] = df.total_qty_oper_login_0 / df.total_qty_oper_login_1.clip(lower=1)
    
    df['same_hash'] = df.item_hash.apply(lambda h: hash_counter[h])
    df['same_name'] = df.processed_name.apply(lambda name: name_counter[name])
    df['same_oper_item_hash'] = df.oper_item_hash.apply(lambda h: oper_item_hash_counter[h])
    
    # df['isnt_last'] = (df.id == df.last_id) * (df.same_hash < 30)
    
    for operator in top_operators:
        df[f'is_{operator}_operator'] = df['index_oper'].replace(' ', '0').astype(float).astype(int) == operator
    
    replace_zeros = ['priority', 'is_privatecategory', 'is_in_yandex', 'mailtype', 'directctg']
    for column in replace_zeros:
        df[column][df[column] == 0] = df[column].mode()[0]
        df[column][df[column] == '0'] = df[column].mode()[0]
    
    df.priority = 1 * (df.priority == 7503.) + 2 * (df.priority == 7504.) + 3 * (df.priority == 7506.)
    
    df.index_oper = df.index_oper.replace(' ', '0').astype(float).astype(int)
    
    for cat_feature in cat_features:
        df[cat_feature] = df[cat_feature].apply(lambda text: str(text))
        
    df['name_mfi_count'] = df.name_mfi.apply(lambda name: 0 if name == '0' else len(name.split(',')))
    
    df.is_return = df.is_return.apply(lambda text: text == 'Y')
        
    df.drop(columns=["oper_type + oper_attr"], inplace=True)
    df.drop(columns=["name_mfi", "item_hash", "processed_name", "oper_item_hash"], inplace=True)
    df.drop(columns=['mailrank'], inplace=True)
    df.drop(columns=useless, inplace=True, errors='ignore')
    
    return df

In [12]:
X_train = prepare_data(X_train)
X_val = prepare_data(X_val)
test_df = prepare_data(test_data)

In [13]:
X_train.to_csv('data/X_train_prepared.csv', index=False)
X_val.to_csv('data/X_val_prepared.csv', index=False)
y_train.to_csv('data/y_train_prepared.csv', index=False)
y_val.to_csv('data/y_val_prepared.csv', index=False)
test_df.to_csv('data/test_df_prepared.csv', index=False)
y_train.to_csv('data/y_train_prepared.csv', index=False)
y_val.to_csv('data/y_val_prepared.csv', index=False)

In [2]:
useless = []
cat_features = ["type", "is_privatecategory", "class", "is_in_yandex", "mailctg", "directctg", "mailtype"]
cat_features = list(set(cat_features) - set(useless))

cat_features = list(set(cat_features) - set(useless))
def to_cat_features(df):
    for cat_feature in cat_features:
        df[cat_feature] = df[cat_feature].apply(lambda text: str(text))
        
X_train = pd.read_csv('data/X_train_prepared.csv')
y_train = pd.read_csv('data/y_train_prepared.csv')
X_val = pd.read_csv('data/X_val_prepared.csv')
y_val = pd.read_csv('data/y_val_prepared.csv')
test_df = pd.read_csv('data/test_df_prepared.csv')
test_ids = pd.read_csv('data/test_ids.csv')


to_cat_features(X_train)
to_cat_features(X_val)
to_cat_features(test_df)

In [3]:
model = CatBoostClassifier(random_state=0, max_depth=8, verbose=10, iterations=100, auto_class_weights="Balanced", cat_features=cat_features)

In [4]:
model.fit(X_train, y_train)

Learning rate set to 0.5
0:	learn: 0.3609663	total: 2.07s	remaining: 3m 25s
10:	learn: 0.2401424	total: 19.6s	remaining: 2m 38s
20:	learn: 0.2311912	total: 36.2s	remaining: 2m 16s
30:	learn: 0.2261218	total: 53.1s	remaining: 1m 58s
40:	learn: 0.2231865	total: 1m 9s	remaining: 1m 40s
50:	learn: 0.2207479	total: 1m 26s	remaining: 1m 23s
60:	learn: 0.2188103	total: 1m 43s	remaining: 1m 6s
70:	learn: 0.2173864	total: 1m 59s	remaining: 49s
80:	learn: 0.2159876	total: 2m 16s	remaining: 32.1s
90:	learn: 0.2147584	total: 2m 33s	remaining: 15.2s
99:	learn: 0.2134527	total: 2m 50s	remaining: 0us


In [5]:
def score_model(model, X, y, threshold=0.5):  # 0.225 is the best for now
    pred = model.predict_proba(X)[:, 1] > threshold
    recall = recall_score(y, pred, average="macro")
    roc_auc = roc_auc_score(y,  pred, multi_class='ovo')
    score = 0.1 * recall + 0.9 * roc_auc
    return score

In [6]:
score_model(model, X_val, y_val, 0.35)

0.899965999306271

In [7]:
score_model(model, X_train, y_train, 0.35) 

0.9047004881621772

In [8]:
sorted(list(zip(X_train.columns, model.feature_importances_)), key = lambda x: -x[1])

[('total_over_person', 12.987636356912972),
 ('same_oper_item_hash', 9.949445437114461),
 ('total_qty_oper_login_1', 9.17107726132425),
 ('specialization_ratio', 7.728569719326381),
 ('mailctg', 7.583457688938222),
 ('unknown_oper_ratio', 6.464988346251635),
 ('index_oper', 5.5666448712818495),
 ('total_qty_over_index_and_type', 4.49698076520119),
 ('transport_pay', 4.398489717528673),
 ('priority', 4.039936673226668),
 ('total_qty_oper_login_0', 3.4692327013826634),
 ('total_qty_over_index', 2.8904327038123956),
 ('same_hash', 2.8620303286911906),
 ('weight', 2.4141984870306583),
 ('price_mfi', 2.3429693830622877),
 ('is_102976_operator', 2.300163818366568),
 ('type', 2.1314514197037573),
 ('class', 1.7548594963484914),
 ('dist_qty_oper_login_1', 1.551437976627038),
 ('name_mfi_count', 1.462353630829835),
 ('is_wrong_phone_number', 1.0672867594450284),
 ('weight_mfi', 1.025901514311994),
 ('same_name', 0.8577896378963729),
 ('last_id', 0.42271869784006705),
 ('is_wrong_rcpn_name', 0.3

In [9]:
test_preds = (model.predict_proba(test_df)[:, 1])
submission = pd.DataFrame({'id': test_ids.id, 'label': test_preds})
submission.to_csv("submission.csv", index=False)

In [10]:
test_preds = (model.predict_proba(test_df)[:, 1] > 0.35).astype(int)
submission = pd.DataFrame({'id': test_ids.id, 'label': test_preds})
submission.to_csv("submission.csv", index=False)

In [4]:
unrounded_sub = pd.read_csv('submission.csv') 
sub = unrounded_sub.copy()
sub.label = (unrounded_sub.label > 0.16).astype(int)
sub.to_csv('submission_rounded.csv', index=False)

In [5]:
print(unrounded_sub.label.mean())
print(sub.label.mean())

0.1372192875755317
0.225103
